In [1]:
#If not installed, install Essentia, mirdata, JAMS, and update other needed libraries. 
# This cell is for running the notebook in Colab
import importlib.util

libraries = ['essentia', 'mirdata', 'jams']

for library in libraries:
    if importlib.util.find_spec(library) is None:
        !pip install {library}

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 56.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 55.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 KB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 KB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.7/90.7 KB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-py3-none-any.whl size=5591954 sha256=c3b13da1533d576610b80cfcdd7738c707ad1d318713b6dd692b97371671327e
  Stored in directory: /root/.cache/

In [2]:
import os

# Imports to support MIR tasks
import mirdata
import jams
import jams.display
import essentia.standard as ess
import librosa
import librosa.display

from torch.utils.data import Dataset
import pandas as pd
import torchaudio

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [8]:
salami = mirdata.initialize("salami")

In [9]:
salami.data_home = '/content/drive/MyDrive/SALAMI'

In [10]:
salami_ids = salami.track_ids  # Return track IDs
salami_data = salami.load_tracks()  # Load Load all tracks in SALAMI

print(type(salami_data[salami_ids[0]])) # Print class
example_track = salami_data[salami_ids[0]]  # Visualize a single track
print(example_track)

<class 'mirdata.datasets.salami.Track'>
Track(
  audio_path="/content/drive/MyDrive/SALAMI/audio/602.mp3",
  sections_annotator1_lowercase_path="...Drive/SALAMI/salami-data-public-hierarchy-corrections/annotations/602/parsed/textfile1_lowercase.txt",
  sections_annotator1_uppercase_path="...Drive/SALAMI/salami-data-public-hierarchy-corrections/annotations/602/parsed/textfile1_uppercase.txt",
  sections_annotator2_lowercase_path="...Drive/SALAMI/salami-data-public-hierarchy-corrections/annotations/602/parsed/textfile2_lowercase.txt",
  sections_annotator2_uppercase_path="...Drive/SALAMI/salami-data-public-hierarchy-corrections/annotations/602/parsed/textfile2_uppercase.txt",
  track_id="602",
  annotator_1_id: ,
  annotator_1_time: ,
  annotator_2_id: ,
  annotator_2_time: ,
  artist: ,
  audio: The track's audio

        Returns,
  broad_genre: ,
  duration: ,
  genre: ,
  sections_annotator_1_lowercase: ,
  sections_annotator_1_uppercase: ,
  sections_annotator_2_lowercase: ,
  sectio

In [19]:
example_track_audio_path = salami_data[salami_ids[0]].audio_path #track's audio
print(f'Path to example track ---> {example_track_audio_path}')

Path to example track ---> /content/drive/MyDrive/SALAMI/audio/602.mp3


In [20]:
audio_dir = '/content/drive/MyDrive/SALAMI/audio'
ann_dir = '/content/drive/MyDrive/SALAMI/salami-data-public-hierarchy-corrections/JAMS'

list_audios = [os.path.join(audio_dir, audio) for audio in os.listdir(audio_dir) if os.path.isfile(os.path.join(audio_dir, audio))]
list_ann = [os.path.join(ann_dir, ann) for ann in os.listdir(ann_dir) if os.path.isfile(os.path.join(ann_dir, ann))]

print(f'List of audio {list_audios}')
print(f'List of annotations {list_ann}')

List of audio ['/content/drive/MyDrive/SALAMI/audio/audio.mp3', '/content/drive/MyDrive/SALAMI/audio/982.mp3', '/content/drive/MyDrive/SALAMI/audio/1640.mp3', '/content/drive/MyDrive/SALAMI/audio/966.mp3', '/content/drive/MyDrive/SALAMI/audio/994.mp3', '/content/drive/MyDrive/SALAMI/audio/1468.mp3', '/content/drive/MyDrive/SALAMI/audio/1482.mp3', '/content/drive/MyDrive/SALAMI/audio/1486.mp3', '/content/drive/MyDrive/SALAMI/audio/1494.mp3', '/content/drive/MyDrive/SALAMI/audio/1496.mp3', '/content/drive/MyDrive/SALAMI/audio/955.mp3', '/content/drive/MyDrive/SALAMI/audio/1483.mp3', '/content/drive/MyDrive/SALAMI/audio/1495.mp3', '/content/drive/MyDrive/SALAMI/audio/957.mp3', '/content/drive/MyDrive/SALAMI/audio/1469.mp3', '/content/drive/MyDrive/SALAMI/audio/1493.mp3', '/content/drive/MyDrive/SALAMI/audio/987.mp3', '/content/drive/MyDrive/SALAMI/audio/980.mp3', '/content/drive/MyDrive/SALAMI/audio/996.mp3', '/content/drive/MyDrive/SALAMI/audio/1018.mp3', '/content/drive/MyDrive/SALAMI/a

In [21]:
def sort_paths_by_last_folder(paths):
    # Split the paths into separate components (directories)
    split_paths = [path.split('/') for path in paths]

    # Extract the last component (directory)
    last_folders = [split_path[-1] for split_path in split_paths]

    # Sort the list based on the extracted last component
    sorted_paths = [path for _, path in sorted(zip(last_folders, paths))]

    return sorted_paths

In [22]:
sorted_audios = sort_paths_by_last_folder(list_audios)
sorted_ann = sort_paths_by_last_folder(list_ann)

print(sorted_audios)
print(sorted_ann)

['/content/drive/MyDrive/SALAMI/audio/10.mp3', '/content/drive/MyDrive/SALAMI/audio/1000.mp3', '/content/drive/MyDrive/SALAMI/audio/1002.mp3', '/content/drive/MyDrive/SALAMI/audio/1003.mp3', '/content/drive/MyDrive/SALAMI/audio/1004.mp3', '/content/drive/MyDrive/SALAMI/audio/1005.mp3', '/content/drive/MyDrive/SALAMI/audio/1006.mp3', '/content/drive/MyDrive/SALAMI/audio/1007.mp3', '/content/drive/MyDrive/SALAMI/audio/1008.mp3', '/content/drive/MyDrive/SALAMI/audio/101.mp3', '/content/drive/MyDrive/SALAMI/audio/1010.mp3', '/content/drive/MyDrive/SALAMI/audio/1011.mp3', '/content/drive/MyDrive/SALAMI/audio/1012.mp3', '/content/drive/MyDrive/SALAMI/audio/1013.mp3', '/content/drive/MyDrive/SALAMI/audio/1014.mp3', '/content/drive/MyDrive/SALAMI/audio/1015.mp3', '/content/drive/MyDrive/SALAMI/audio/1016.mp3', '/content/drive/MyDrive/SALAMI/audio/1018.mp3', '/content/drive/MyDrive/SALAMI/audio/1019.mp3', '/content/drive/MyDrive/SALAMI/audio/1020.mp3', '/content/drive/MyDrive/SALAMI/audio/1021.

In [23]:
for file1, file2 in zip(sorted_audios, sorted_ann):
    print(file1, file2)

/content/drive/MyDrive/SALAMI/audio/10.mp3 /content/drive/MyDrive/SALAMI/salami-data-public-hierarchy-corrections/JAMS/10.jams
/content/drive/MyDrive/SALAMI/audio/1000.mp3 /content/drive/MyDrive/SALAMI/salami-data-public-hierarchy-corrections/JAMS/100.jams
/content/drive/MyDrive/SALAMI/audio/1002.mp3 /content/drive/MyDrive/SALAMI/salami-data-public-hierarchy-corrections/JAMS/1000.jams
/content/drive/MyDrive/SALAMI/audio/1003.mp3 /content/drive/MyDrive/SALAMI/salami-data-public-hierarchy-corrections/JAMS/1002.jams
/content/drive/MyDrive/SALAMI/audio/1004.mp3 /content/drive/MyDrive/SALAMI/salami-data-public-hierarchy-corrections/JAMS/1003.jams
/content/drive/MyDrive/SALAMI/audio/1005.mp3 /content/drive/MyDrive/SALAMI/salami-data-public-hierarchy-corrections/JAMS/1004.jams
/content/drive/MyDrive/SALAMI/audio/1006.mp3 /content/drive/MyDrive/SALAMI/salami-data-public-hierarchy-corrections/JAMS/1005.jams
/content/drive/MyDrive/SALAMI/audio/1007.mp3 /content/drive/MyDrive/SALAMI/salami-data-p

In [ ]:
for index, track in enumerate(salami_data):
  print(f'Iteration --> {index}', 
        f'Track data --> {salami_data[salami_ids[int(index)]]}', 
        f'JAMS annotaion --> {salami_data[salami_ids[int(index)]].to_jams()}')

Iteration --> 0 Track data --> Track(
  audio_path="/content/drive/MyDrive/SALAMI/audio/602.mp3",
  sections_annotator1_lowercase_path="...Drive/SALAMI/salami-data-public-hierarchy-corrections/annotations/602/parsed/textfile1_lowercase.txt",
  sections_annotator1_uppercase_path="...Drive/SALAMI/salami-data-public-hierarchy-corrections/annotations/602/parsed/textfile1_uppercase.txt",
  sections_annotator2_lowercase_path="...Drive/SALAMI/salami-data-public-hierarchy-corrections/annotations/602/parsed/textfile2_lowercase.txt",
  sections_annotator2_uppercase_path="...Drive/SALAMI/salami-data-public-hierarchy-corrections/annotations/602/parsed/textfile2_uppercase.txt",
  track_id="602",
  annotator_1_id: ,
  annotator_1_time: ,
  annotator_2_id: ,
  annotator_2_time: ,
  artist: ,
  audio: The track's audio

        Returns,
  broad_genre: ,
  duration: ,
  genre: ,
  sections_annotator_1_lowercase: ,
  sections_annotator_1_uppercase: ,
  sections_annotator_2_lowercase: ,
  sections_annota

Iteration --> 1 Track data --> Track(
  audio_path="/content/drive/MyDrive/SALAMI/audio/488.mp3",
  sections_annotator1_lowercase_path="...Drive/SALAMI/salami-data-public-hierarchy-corrections/annotations/488/parsed/textfile1_lowercase.txt",
  sections_annotator1_uppercase_path="...Drive/SALAMI/salami-data-public-hierarchy-corrections/annotations/488/parsed/textfile1_uppercase.txt",
  sections_annotator2_lowercase_path="...Drive/SALAMI/salami-data-public-hierarchy-corrections/annotations/488/parsed/textfile2_lowercase.txt",
  sections_annotator2_uppercase_path="...Drive/SALAMI/salami-data-public-hierarchy-corrections/annotations/488/parsed/textfile2_uppercase.txt",
  track_id="488",
  annotator_1_id: ,
  annotator_1_time: ,
  annotator_2_id: ,
  annotator_2_time: ,
  artist: ,
  audio: The track's audio

        Returns,
  broad_genre: ,
  duration: ,
  genre: ,
  sections_annotator_1_lowercase: ,
  sections_annotator_1_uppercase: ,
  sections_annotator_2_lowercase: ,
  sections_annota

Iteration --> 2 Track data --> Track(
  audio_path="/content/drive/MyDrive/SALAMI/audio/1477.mp3",
  sections_annotator1_lowercase_path="...rive/SALAMI/salami-data-public-hierarchy-corrections/annotations/1477/parsed/textfile1_lowercase.txt",
  sections_annotator1_uppercase_path="...rive/SALAMI/salami-data-public-hierarchy-corrections/annotations/1477/parsed/textfile1_uppercase.txt",
  sections_annotator2_lowercase_path=None,
  sections_annotator2_uppercase_path=None,
  track_id="1477",
  annotator_1_id: ,
  annotator_1_time: ,
  annotator_2_id: ,
  annotator_2_time: ,
  artist: ,
  audio: The track's audio

        Returns,
  broad_genre: ,
  duration: ,
  genre: ,
  sections_annotator_1_lowercase: ,
  sections_annotator_1_uppercase: ,
  sections_annotator_2_lowercase: ,
  sections_annotator_2_uppercase: ,
  source: ,
  title: ,
) JAMS annotaion --> {
  "annotations": [
    {
      "annotation_metadata": {
        "curator": {
          "name": "",
          "email": ""
        },
   

Iteration --> 3 Track data --> Track(
  audio_path="/content/drive/MyDrive/SALAMI/audio/787.mp3",
  sections_annotator1_lowercase_path="...Drive/SALAMI/salami-data-public-hierarchy-corrections/annotations/787/parsed/textfile1_lowercase.txt",
  sections_annotator1_uppercase_path="...Drive/SALAMI/salami-data-public-hierarchy-corrections/annotations/787/parsed/textfile1_uppercase.txt",
  sections_annotator2_lowercase_path="...Drive/SALAMI/salami-data-public-hierarchy-corrections/annotations/787/parsed/textfile2_lowercase.txt",
  sections_annotator2_uppercase_path="...Drive/SALAMI/salami-data-public-hierarchy-corrections/annotations/787/parsed/textfile2_uppercase.txt",
  track_id="787",
  annotator_1_id: ,
  annotator_1_time: ,
  annotator_2_id: ,
  annotator_2_time: ,
  artist: ,
  audio: The track's audio

        Returns,
  broad_genre: ,
  duration: ,
  genre: ,
  sections_annotator_1_lowercase: ,
  sections_annotator_1_uppercase: ,
  sections_annotator_2_lowercase: ,
  sections_annota

Iteration --> 4 Track data --> Track(
  audio_path="/content/drive/MyDrive/SALAMI/audio/911.mp3",
  sections_annotator1_lowercase_path="...Drive/SALAMI/salami-data-public-hierarchy-corrections/annotations/911/parsed/textfile1_lowercase.txt",
  sections_annotator1_uppercase_path="...Drive/SALAMI/salami-data-public-hierarchy-corrections/annotations/911/parsed/textfile1_uppercase.txt",
  sections_annotator2_lowercase_path="...Drive/SALAMI/salami-data-public-hierarchy-corrections/annotations/911/parsed/textfile2_lowercase.txt",
  sections_annotator2_uppercase_path="...Drive/SALAMI/salami-data-public-hierarchy-corrections/annotations/911/parsed/textfile2_uppercase.txt",
  track_id="911",
  annotator_1_id: ,
  annotator_1_time: ,
  annotator_2_id: ,
  annotator_2_time: ,
  artist: ,
  audio: The track's audio

        Returns,
  broad_genre: ,
  duration: ,
  genre: ,
  sections_annotator_1_lowercase: ,
  sections_annotator_1_uppercase: ,
  sections_annotator_2_lowercase: ,
  sections_annota

...

Iteration --> 21 Track data --> Track(
  audio_path="/content/drive/MyDrive/SALAMI/audio/12.mp3",
  sections_annotator1_lowercase_path="...yDrive/SALAMI/salami-data-public-hierarchy-corrections/annotations/12/parsed/textfile1_lowercase.txt",
  sections_annotator1_uppercase_path="...yDrive/SALAMI/salami-data-public-hierarchy-corrections/annotations/12/parsed/textfile1_uppercase.txt",
  sections_annotator2_lowercase_path="...yDrive/SALAMI/salami-data-public-hierarchy-corrections/annotations/12/parsed/textfile2_lowercase.txt",
  sections_annotator2_uppercase_path="...yDrive/SALAMI/salami-data-public-hierarchy-corrections/annotations/12/parsed/textfile2_uppercase.txt",
  track_id="12",
  annotator_1_id: ,
  annotator_1_time: ,
  annotator_2_id: ,
  annotator_2_time: ,
  artist: ,
  audio: The track's audio

        Returns,
  broad_genre: ,
  duration: ,
  genre: ,
  sections_annotator_1_lowercase: ,
  sections_annotator_1_uppercase: ,
  sections_annotator_2_lowercase: ,
  sections_annotat

Iteration --> 22 Track data --> Track(
  audio_path="/content/drive/MyDrive/SALAMI/audio/710.mp3",
  sections_annotator1_lowercase_path="...Drive/SALAMI/salami-data-public-hierarchy-corrections/annotations/710/parsed/textfile1_lowercase.txt",
  sections_annotator1_uppercase_path="...Drive/SALAMI/salami-data-public-hierarchy-corrections/annotations/710/parsed/textfile1_uppercase.txt",
  sections_annotator2_lowercase_path=None,
  sections_annotator2_uppercase_path=None,
  track_id="710",
  annotator_1_id: ,
  annotator_1_time: ,
  annotator_2_id: ,
  annotator_2_time: ,
  artist: ,
  audio: The track's audio

        Returns,
  broad_genre: ,
  duration: ,
  genre: ,
  sections_annotator_1_lowercase: ,
  sections_annotator_1_uppercase: ,
  sections_annotator_2_lowercase: ,
  sections_annotator_2_uppercase: ,
  source: ,
  title: ,
) JAMS annotaion --> {
  "annotations": [
    {
      "annotation_metadata": {
        "curator": {
          "name": "",
          "email": ""
        },
    

Iteration --> 24 Track data --> Track(
  audio_path="/content/drive/MyDrive/SALAMI/audio/960.mp3",
  sections_annotator1_lowercase_path="...Drive/SALAMI/salami-data-public-hierarchy-corrections/annotations/960/parsed/textfile1_lowercase.txt",
  sections_annotator1_uppercase_path="...Drive/SALAMI/salami-data-public-hierarchy-corrections/annotations/960/parsed/textfile1_uppercase.txt",
  sections_annotator2_lowercase_path="...Drive/SALAMI/salami-data-public-hierarchy-corrections/annotations/960/parsed/textfile2_lowercase.txt",
  sections_annotator2_uppercase_path="...Drive/SALAMI/salami-data-public-hierarchy-corrections/annotations/960/parsed/textfile2_uppercase.txt",
  track_id="960",
  annotator_1_id: ,
  annotator_1_time: ,
  annotator_2_id: ,
  annotator_2_time: ,
  artist: ,
  audio: The track's audio

        Returns,
  broad_genre: ,
  duration: ,
  genre: ,
  sections_annotator_1_lowercase: ,
  sections_annotator_1_uppercase: ,
  sections_annotator_2_lowercase: ,
  sections_annot

Iteration --> 26 Track data --> Track(
  audio_path="/content/drive/MyDrive/SALAMI/audio/915.mp3",
  sections_annotator1_lowercase_path="...Drive/SALAMI/salami-data-public-hierarchy-corrections/annotations/915/parsed/textfile1_lowercase.txt",
  sections_annotator1_uppercase_path="...Drive/SALAMI/salami-data-public-hierarchy-corrections/annotations/915/parsed/textfile1_uppercase.txt",
  sections_annotator2_lowercase_path="...Drive/SALAMI/salami-data-public-hierarchy-corrections/annotations/915/parsed/textfile2_lowercase.txt",
  sections_annotator2_uppercase_path="...Drive/SALAMI/salami-data-public-hierarchy-corrections/annotations/915/parsed/textfile2_uppercase.txt",
  track_id="915",
  annotator_1_id: ,
  annotator_1_time: ,
  annotator_2_id: ,
  annotator_2_time: ,
  artist: ,
  audio: The track's audio

        Returns,
  broad_genre: ,
  duration: ,
  genre: ,
  sections_annotator_1_lowercase: ,
  sections_annotator_1_uppercase: ,
  sections_annotator_2_lowercase: ,
  sections_annot

ValueError: ignored

In [ ]:
#Create df here

In [ ]:
class SalamiDatasetLoader(Dataset):

    def __init__(self, annotations_file, audio_dir):
        self.annotations = pd.read_csv(annotations_file)
        self.audio_dir = audio_dir

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        audio_sample_path = self._get_audio_sample_path(index)
        label = self._get_audio_sample_label(index)
        signal, sr = torchaudio.load(audio_sample_path)
        return signal, label

    def _get_audio_sample_path(self, index):
        fold = f"fold{self.annotations.iloc[index, 5]}"
        path = os.path.join(self.audio_dir, fold, self.annotations.iloc[
            index, 0])
        return path

In [ ]:
#ANNOTATIONS_FILE = "XXXX.csv"
AUDIO_DIR = '/content/drive/MyDrive/SALAMI/audio'

In [ ]:
#SDL = SalamiDatasetLoader(ANNOTATIONS_FILE, AUDIO_DIR)

In [ ]:
#print(f"There are {len(SDL)} samples in the dataset.")

In [ ]:
#signal, label = SDL[0]